<a href="https://colab.research.google.com/github/geniusrusho/Xgboost-NLTKZ-TF-IDF/blob/main/Xgboost%2CNLTKZ%26TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.4/210.4 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 KB 11.7 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=b0959fe4bcf9e6b73520afee62be509af6dda5ce3af180ca576c60ac8cb4547d
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  A

In [ ]:
import optuna


In [ ]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/divyansha1115/Graduate-Admission-Prediction/master/Admission_Predict.csv')

In [ ]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91
398,399,312,103,3,3.5,4.0,8.78,0,0.67


In [ ]:
x= df.drop(columns=['Serial No.'])
x

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...
395,324,110,3,3.5,3.5,9.04,1,0.82
396,325,107,3,3.0,3.5,9.11,1,0.84
397,330,116,4,5.0,4.5,9.45,1,0.91
398,312,103,3,3.5,4.0,8.78,0,0.67


In [ ]:
y=df['Chance of Admit ']

In [ ]:
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
395    0.82
396    0.84
397    0.91
398    0.67
399    0.95
Name: Chance of Admit , Length: 400, dtype: float64

In [98]:
from sklearn.metrics import mean_squared_error
from inspect import Parameter
def objective(trail, data = x ,target= y ):
  train_x,test_x,train_y,test_y = train_test_split(data , target , test_size = 0.10,random_state = 30)
  param = {
      
      'train_method': 'gpu_hist',
      'lambda':trail.suggest_loguniform('lambda', 1e-4,10.0),
      
      'alpha':trail.suggest_loguniform('alpha',1e-4,10.0),
      'colsample_bytree' :trail.suggest_catagorical('colsample_bytree', [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]),
       'subsample' :trail.suggest_catagorical('subsample', [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]),
       'learning_rate' :trail.suggest_catagorical('learning_rate', [.00001,.0003,.008,.02,.01,1,10,20]),
       'n_estimator' :3000,
       'max_depth' :trail.suggest_catagorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
       'random_state' :trail.suggest_catagorical('random_state' , [10,20,30,2000,3454,243123]),
       'min_child_weight' :trail.suggest_int('min_child_weight',1,2000),
       'virbosity' :3,

    }
  xgb_reg_model = xgb.XGBRegressor(**param)
  xgb_reg_model.fit(train_x,train_y,eval_set = [(test_x,test_y)], verbose=True)
  pred_xgb = xgb.reg_model.predict(test_x)
  rmse = mean_squared_error(test_y,pred_xgb)
  return rmse





      
  

In [97]:
find_param=optuna.create_study(direction='minimize')


[I 2023-01-04 17:33:59,694] A new study created in memory with name: no-name-cfac68af-aaec-462a-bad0-e498df770f5d
[W 2023-01-04 17:33:59,699] Trial 0 failed because of the following error: ValueError('Found input variables with inconsistent numbers of samples: [400, 100]')
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-91-380244e7f97e>", line 4, in objective
    train_x,test_x,train_y,test_y = train_test_split(data , target , test_size = 0.10,random_state = 30)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py", line 2417, in train_test_split
    arrays = indexable(*arrays)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py", line 378, in indexable
    check_consistent_length(*result)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py", line 332, in check_consistent_

ValueError: ignored

In [ ]:
xgb.XGBRFRegressor()

XGBRFRegressor()